In [1]:
import os
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend\\research'

In [2]:
os.chdir("..")
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/dewasahu2003/endtoend.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "dewasahu2003"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "6976e6dedfc675e1145823597f708aa2b40572a2"


In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvalConfig:
    root_dir:Path
    test_data_path:Path
    model_path:Path
    metric_file:str
    
    all_params:dict
    target_column:str
    
    mlflow_uri:str
    

In [10]:
from src.datasci.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH,
                                   SCHEMA_FILE_PATH)
from src.datasci.utils.common import create_directories, read_yaml, save_json


class ConfiguratonManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_model_eval_config(self)-> ModelEvalConfig:
        config = self.config.model_evaluation
        param = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        
        model_eval_config = ModelEvalConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file=config.metric_file,
            all_params=param,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/dewasahu2003/endtoend.mlflow")
        return model_eval_config

In [13]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from urllib.parse import urlparse
import mlflow as mf
import mlflow.sklearn as mfsk
import joblib

class ModelEval:
    def __init__(self,config:ModelEvalConfig):
        self.config =  config
    
    def eval_metrics(self,actual,pred):
        rmse =  np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual,pred)
        r2 = r2_score(actual,pred)
        return rmse,mae,r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column],axis=1)
        test_y = test_data[[self.config.target_column]]
        
        mf.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mf.get_tracking_uri()).scheme
        
        with mf.start_run():
            predicted_qualities = model.predict(test_x)
            (rmse,mae,r2) = self.eval_metrics(test_y,predicted_qualities)
            # Log parameter, metrics, and model to MLflow
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file),data=scores)
            
            mf.log_params(self.config.all_params)
            mf.log_metrics(scores)
            
            if tracking_url_type_store != "file":
                #register the model
                mfsk.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mfsk.log_model(model, "model")
    

In [14]:
try:
    config = ConfiguratonManager()
    model_config = config.get_model_eval_config()
    model_eval = ModelEval(model_config)
    model_eval.log_into_mlflow()
except Exception as e:
    raise e
    

[2024-12-07 16:17:24,389: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-07 16:17:24,391: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-07 16:17:24,394: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-07 16:17:24,396: INFO: common: created directory at: artifacts]
[2024-12-07 16:17:24,397: INFO: common: created directory at: artifacts/model_evaluation]
[2024-12-07 16:17:25,383: INFO: common: json file saved at: artifacts\model_evaluation\metrics.jsons]


2024/12/07 16:17:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/12/07 16:17:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.


🏃 View run able-goose-436 at: https://dagshub.com/dewasahu2003/endtoend.mlflow/#/experiments/0/runs/f991e77dec8b4a8fb70fa67e6d407bba
🧪 View experiment at: https://dagshub.com/dewasahu2003/endtoend.mlflow/#/experiments/0
